<a href="https://colab.research.google.com/github/alaaguedda/product_sales/blob/main/saudi_product_sales_compitition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])

df.head()

In [14]:
df.columns = ['Date','Product','City','Sales']
df['Product'] = df['Product'].str.replace('منتج ','')
top3_sales_product = df.groupby('Product')['Sales'].sum().sort_values(ascending=False).nlargest(3)
top3_sales_product

In [ ]:
# 1. Total sales per day
daily_sales = df.groupby('Date', as_index=False)['Sales'].sum()
daily_sales.columns = ['Date', 'Total_Sales']

daily_sales

In [ ]:
# 2. Top 3 products with sales
top_products_daily = (
    df.groupby(['Date', 'Product'], as_index=False)['Sales']
      .sum()
      .sort_values(['Date', 'Sales'], ascending=[True, False])
)
top_products_daily

In [ ]:


# 3. Aggregate into two separate lists (products + sales)
top3_daily = (
    top_products_daily.groupby('Date')
    .apply(lambda x: pd.Series({
        'Top3_Products': x.nlargest(3, 'Sales')['Product'].tolist(),
        'Top3_Product_Sales': x.nlargest(3, 'Sales')['Sales'].tolist()
    }))
    .reset_index()
)
top3_daily


In [ ]:
# 4. Merge with totals
df_sales_optimized = daily_sales.merge(top3_daily, on='Date')
df_sales_optimized

In [ ]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

# Use only Total_Sales for anomaly detection
X = df_sales_optimized[['Total_Sales']]

# Train Isolation Forest
iso = IsolationForest(contamination=0.05, random_state=42)
df_sales_optimized['anomaly'] = iso.fit_predict(X)

# -1 = anomaly, 1 = normal
df_sales_optimized['is_anomaly'] = df_sales_optimized['anomaly'] == -1

# Plot
plt.figure(figsize=(12,6))
plt.plot(df_sales_optimized['Date'], df_sales_optimized['Total_Sales'], label="Total Sales")
plt.scatter(
    df_sales_optimized['Date'][df_sales_optimized['is_anomaly']],
    df_sales_optimized['Total_Sales'][df_sales_optimized['is_anomaly']],
    color='red', label="Anomaly"
)
plt.legend()
plt.title("Daily Sales with Anomalies")
plt.xticks(rotation=45)
plt.show()


In [ ]:
!pip install mlxtend

from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd


In [ ]:

# Explode Top3_Products column (so each row = Date, Product)
exploded = df_sales_optimized[['Date','Top3_Products']].explode('Top3_Products')

# Create basket-like pivot table: one row per date, one col per product (1 if product in top3)
basket = pd.crosstab(exploded['Date'], exploded['Top3_Products'])

# Find frequent itemsets
frequent_itemsets = apriori(basket, min_support=0.2, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Sort strongest rules
rules = rules.sort_values(by='lift', ascending=False)
rules.head(10)


In [13]:
df['Region'].unique()

array(['الرياض', 'جدة', 'الخبر', 'مكة', 'الدمام'], dtype=object)

In [20]:
df.head(10)

,Date,Product,City,Sales
0,2025-01-01,A,الرياض,4039
1,2025-01-01,B,جدة,3997
2,2025-01-01,C,الخبر,2645
3,2025-01-01,D,جدة,4754
4,2025-01-01,E,جدة,3568
5,2025-01-02,A,الرياض,1341
6,2025-01-02,B,مكة,1802
7,2025-01-02,C,الدمام,1686
8,2025-01-02,D,الخبر,2606
9,2025-01-02,E,جدة,3384


In [21]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# --- Use your processed df_sales_optimized ---
df_sales_optimized['Date'] = pd.to_datetime(df_sales_optimized['Date'])

# Start app
app = dash.Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("📊 Sales Dashboard", style={'text-align':'center'}),

    # Date filter
    dcc.DatePickerRange(
        id='date_range',
        start_date=df_sales_optimized['Date'].min(),
        end_date=df_sales_optimized['Date'].max()
    ),

    # Daily Sales Chart
    dcc.Graph(id='daily_sales_chart'),

    # Top 3 Products Chart
    dcc.Graph(id='top_products_chart'),

    # Anomalies Highlight
    dcc.Graph(id='anomaly_chart'),

    # Association Rules Table
    html.H3("Association Rules"),
    html.Div(id="rules_table")
])

# Callbacks
@app.callback(
    [Output('daily_sales_chart','figure'),
     Output('top_products_chart','figure'),
     Output('anomaly_chart','figure'),
     Output('rules_table','children')],
    [Input('date_range','start_date'),
     Input('date_range','end_date')]
)
def update_dashboard(start_date, end_date):
    # Filter by date
    mask = (df_sales_optimized['Date'] >= start_date) & (df_sales_optimized['Date'] <= end_date)
    df_filtered = df_sales_optimized.loc[mask]

    # --- Chart 1: Daily Sales ---
    fig1 = px.line(df_filtered, x="Date", y="Total_Sales", title="Total Sales Over Time")

    # --- Chart 2: Top 3 Products ---
    exploded = df_filtered[['Date','Top3_Products']].explode('Top3_Products')
    fig2 = px.histogram(exploded, x="Top3_Products", title="Top 3 Products Frequency")

    # --- Chart 3: Anomalies ---
    fig3 = px.line(df_filtered, x="Date", y="Total_Sales", title="Anomaly Detection")
    anomalies = df_filtered[df_filtered['is_anomaly']]
    fig3.add_scatter(x=anomalies['Date'], y=anomalies['Total_Sales'],
                     mode='markers', marker=dict(color='red', size=10), name="Anomaly")

    # --- Table: Association Rules (top 10) ---
    from mlxtend.frequent_patterns import apriori, association_rules
    exploded = df_filtered[['Date','Top3_Products']].explode('Top3_Products')
    basket = pd.crosstab(exploded['Date'], exploded['Top3_Products'])
    frequent_itemsets = apriori(basket, min_support=0.2, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values("lift", ascending=False).head(10)

    table = html.Table([
        html.Tr([html.Th(col) for col in rules.columns])] +
        [html.Tr([html.Td(str(rules.iloc[i][col])) for col in rules.columns]) for i in range(len(rules))]
    )

    return fig1, fig2, fig3, table

if __name__ == '__main__':
    app.run_server(debug=True)


ModuleNotFoundError: No module named 'dash'